



Built using guidance from https://arxiv.org/pdf/1512.09300.pdf

Features:
  * Uses ELU activations
  * Deconvolution uses upscaling unpool layer before affine operator, rather than spacing with zeros
  * Batch normalization after each transformation
  * Dropout layer after activation
  * abs-sum image loss rather than cross-entropy loss

# Summary

Best result so far is 10 epochs of the first training batch, with the prior and the prediction weighted equally.

Running another 10 epochs doesn't hurt the similarity results, but it does make the reconstructions worse.

Not quite as good after 10 epochs with regularization=0.1 (down-weighted prior)

In [1]:
import tensorflow as tf
 
print("running TensorFlow version {}".format(tf.__version__))

running TensorFlow version 1.8.0


In [2]:
# Control memory usage

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
# Report OOM details

run_options = tf.RunOptions(report_tensor_allocations_upon_oom = True)


# Import data

## Option 1: read from tfrecords

In [4]:
import os
import re

from operator import itemgetter

In [5]:
LOG_ROOT = '../../data/'

RUN_NAME = 'cvae/vae_004'
SUMMARY_DIR = os.path.join(LOG_ROOT, 'logs', RUN_NAME)
MODEL_DIR = os.path.join(LOG_ROOT, 'models', RUN_NAME)
MODEL_GRAPH = os.path.join(MODEL_DIR, 'vae.meta')
MODEL_PREFIX = os.path.join(MODEL_DIR, 'vae')

DATA_SIZE = 'all_packs'
EXPT_NAME = 'expt_004'
VIS_NAME = 'vis_001'
DATA_ROOT = '/var/data/processed'

EXPT_DIR = os.path.join(DATA_ROOT, EXPT_NAME, 'data')
TFRECORDS_DIR = os.path.join('/var/data/original/tfrecords/', DATA_SIZE)
EMBEDDINGS_DIR = os.path.join(EXPT_DIR, 'embeddings')
ENCODED_IMAGES_DIR =  os.path.join(EXPT_DIR, 'images')

In [6]:
train_256_pattern = re.compile('^train_(?P<block_id>[0-9]{3}).tfrecords')
validate_256_pattern = re.compile('^validate_(?P<block_id>[0-9]{3}).tfrecords')
test_256_pattern = re.compile('^test_(?P<block_id>[0-9]{3}).tfrecords')
golden_256_pattern = re.compile('golden_(?P<block_id>[0-9]{3}).tfrecords')


In [7]:

ALL_TFRECORDS = os.listdir(TFRECORDS_DIR)
def get_sorted_records(pattern, directory):
     return [ \
         os.path.join(directory, _file) \
         for _file in \
         sorted([_m[0] for _m in \
             [pattern.match(_f) for _f in os.listdir(directory)] if _m]) \
     ]

GOLDEN_TFRECORDS = get_sorted_records(golden_256_pattern, TFRECORDS_DIR)
TRAIN_TFRECORDS = get_sorted_records(train_256_pattern, TFRECORDS_DIR)
VALIDATE_TFRECORDS = get_sorted_records(validate_256_pattern, TFRECORDS_DIR)
TEST_TFRECORDS = get_sorted_records(test_256_pattern, TFRECORDS_DIR)

In [8]:
TRAIN_TFRECORDS, VALIDATE_TFRECORDS,GOLDEN_TFRECORDS, TEST_TFRECORDS

(['/var/data/original/tfrecords/all_packs/train_003.tfrecords'],
 ['/var/data/original/tfrecords/all_packs/validate_003.tfrecords'],
 ['/var/data/original/tfrecords/all_packs/golden_003.tfrecords'],
 ['/var/data/original/tfrecords/all_packs/test_003.tfrecords'])

## Option 2: read from files

In [ ]:
def _load_image(path, size):
    ''' load grayscale image from file; resize if necessary
    '''
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        raise ValueError("empty image: {}".format(path))

    if size:
        img = cv2.resize(img, (size,size), interpolation=cv2.INTER_CUBIC)

    img = img.astype(np.float32)
    return img

In [ ]:
IMAGES_DIR = '/var/data/original/data/images'
TEST_JSON = '/var/data/original/labels/test_data.json'
VALIDATE_JSON = '/var/data/original/labels/validate_data.json'
TRAIN_JSON = '/var/data/original/labels/train_data.json'

In [ ]:
import json
import glob
import cv2
from collections import defaultdict

def gen_images(images_dir, json_dir):
    
    images = []
    filenames = []
    labels = []
    with open(json_dir, 'r') as fi:
        records = json.load(fi)
        
    json_filenames = []
    filenames2labels = defaultdict(list)


    for r in records:
        json_filenames.append(r['filename'])
        filenames2labels[r['filename']].append(r['labels'])

    for image_path in glob.glob(os.path.join(images_dir,'*')):
        filename = image_path[-16:]
        if(filename in json_filenames):
            filenames.append(filename)
            labels.append(filenames2labels[filename][0])
            images.append(_load_image(image_path, 256))
            
    return filenames, images, labels

In [ ]:
train_filenames, train_images, train_labels = gen_images(IMAGES_DIR, TRAIN_JSON)

In [ ]:
print(len(train_images))

## Continue

In [9]:
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)

In [1]:
def _decode(serialized_example):
    '''Parses an image and label from the given `serialized_example`
    '''
    features = tf.parse_single_example(
        serialized_example,
        features={
            'filename': tf.FixedLenFeature([], tf.string),
            'image': tf.FixedLenFeature([], tf.string),
            #'view': tf.FixedLenFeature([], tf.string),
            #'gender': tf.FixedLenFeature([], tf.string),
            #'age': tf.FixedLenFeature([], tf.int64),
            'labels': tf.FixedLenSequenceFeature( [], dtype=tf.int64, default_value=-1,allow_missing=True)
            })
       

    # Convert from a scalar string tensor
    filename = tf.cast(features['filename'], tf.string)
    image = tf.decode_raw(features['image'], tf.float32)
    #view = tf.cast(features['view'], tf.string)
    #gender = tf.cast(features['gender'], tf.string)
    #age = tf.cast(features['age'], tf.int32)
    labels = tf.cast(features['labels'], tf.int32)
    
    return filename, image, labels

def _filter(filename, image, labels):
    
    sub_string = tf.substr(filename,3,1)
    return tf.equal(sub_string, "0")

def _augment(filename, image, labels):
    '''Placeholder for data augmentation
    '''
    image = tf.reshape(image, [256, 256, 1])
    return filename, image, labels


def _normalize(filename, image, labels):
    '''Convert `image` from [0, 255] -> [-0.5, 0.5] floats
    '''
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return filename, image, labels

In [11]:
def inputs(filenames, batch_size, num_epochs, num_shards, shard_index):
    ''' Reads input data num_epochs times or forever if num_epochs is None
        returns dataset, iterator pair
    '''

    with tf.name_scope('input'):
        # TFRecordDataset opens a binary file and reads one record at a time.
        # `filename` could also be a list of filenames, which will be read in order.
       
        
        dataset = tf.data.TFRecordDataset(filenames)
        
        # The map transformation takes a function and applies it to every element
        # of the dataset.
        
        dataset = dataset.map(_decode)
        dataset = dataset.shard(num_shards, shard_index)
        #dataset = dataset.filter(_filter)
        dataset = dataset.map(_augment)
        dataset = dataset.map(_normalize)

        # The shuffle transformation uses a finite-sized buffer to shuffle elements
        # in memory. The parameter is the number of elements in the buffer. For
        # completely uniform shuffling, set the parameter to be the same as the
        # number of elements in the dataset.
        dataset = dataset.shuffle(1000 + 3 * batch_size)

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)

        iterator = dataset.make_one_shot_iterator()

    return dataset, iterator

# VAE Model

In [12]:
# training parameters
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = 30

DROPOUT = 0.7
REGULARIZATION = 0.1

DISPLAY_EVERY = 100

## Loggers

In [13]:
def variable_summary(x, name):
    with tf.variable_scope(name):
        mean = tf.reduce_mean(x)
        tf.summary.scalar('mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(x - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(x))
        tf.summary.scalar('min', tf.reduce_min(x))
        tf.summary.histogram('histogram', x)

## Component layers

In [14]:
# unpool operation doesn't yet exist in TF

def unpool_op(x, stride, name='unpool'):

    with tf.name_scope(name) as scope:

        if stride==1:
            return x

        shape = x.get_shape().as_list()
        dim = len(shape[1:-1])
        out = (tf.reshape(x, [-1] + shape[-dim:]))
        for i in range(dim, 0, -1):
            out = tf.concat([out]*stride, i)
        out_size = [-1] + [s * stride for s in shape[1:-1]] + [shape[-1]]
        out = tf.reshape(out, out_size, name=scope)
    return out

In [15]:
def convolution_layer(x, dims, stride, train, bias=None, name='conv', activation=tf.nn.elu):

    with tf.variable_scope(name):

        # Parameters
        weights = tf.get_variable('w', dims,
                    initializer=tf.contrib.layers.xavier_initializer())

        if bias is not None:
            biases = tf.get_variable('b', [dims[-1]],
                        initializer=tf.random_normal_initializer())

        # Layer structure
        if bias is None:
             conv = tf.nn.conv2d(x, weights, strides=[1, stride, stride, 1], padding='SAME', name='conv')
        else:
             conv = tf.nn.bias_add(tf.nn.conv2d(x, weights, strides=[1, stride, stride, 1],
                        padding='SAME'), biases, name='conv')
        normalized = tf.layers.batch_normalization(conv, axis=3, training=train,
                    name='spatial_batch_norm')        
        activations = activation(normalized, name='activation')
        activations = tf.layers.dropout(activations, rate=DROPOUT, training=train, name='dropout')

        # Variable summaries
        variable_summary(weights, 'weights')
        if bias is not None:
            variable_summary(biases, 'biases')
        tf.summary.histogram('pre-activations', normalized)
        tf.summary.histogram('activations', activations)

        return activations

In [16]:
def deconvolution_layer(x, dims, stride, train, bias=None, name='deconv', activation=tf.nn.elu):

    with tf.variable_scope(name):

        # Parameters
        weights = tf.get_variable('w', dims,
                    initializer=tf.contrib.layers.xavier_initializer())
        if bias is not None:
            biases = tf.get_variable('b', [dims[-1]],
                        initializer=tf.random_normal_initializer())

        # Layer structure
        unpool = unpool_op(x, stride, name='unpool')
        if bias is None:
            deconv = tf.nn.conv2d(unpool, weights, strides=[1, 1, 1, 1], padding='SAME', name='deconv')
        else:
            deconv = tf.nn.bias_add(tf.nn.conv2d(unpool, weights, strides=[1, 1, 1, 1],
                        padding='SAME'), biases, name='deconv')
        normalized = tf.layers.batch_normalization(deconv, axis=3, training=train,
                    name='spatial_batch_norm')        
        activations = activation(deconv, name='activation')
        activations = tf.layers.dropout(activations, rate=DROPOUT, training=train, name='dropout')

        # Variable summaries
        variable_summary(weights, 'weights')
        if bias is not None:
            variable_summary(biases, 'biases')
        tf.summary.histogram('pre-activations', deconv)
        tf.summary.histogram('activations', activations)

        return activations

In [17]:
def dense_layer(x, dims, train, bias=None, name='fc', activation=tf.nn.elu):

    with tf.variable_scope(name):

        # Parameters
        weights = tf.get_variable('w', dims,
                    initializer=tf.contrib.layers.xavier_initializer())
        if bias is not None:
            biases = tf.get_variable('b', [dims[-1]],
                        initializer=tf.random_normal_initializer())

        # Layer structure
        if bias is None:
            dense = tf.matmul(x, weights, name='dense')
        else:
            dense = tf.nn.bias_add(tf.matmul(x, weights), biases, name='dense')

        normalized = tf.layers.batch_normalization(dense, axis=1, training=train,
                    name='batch_norm')
        activations = activation(normalized, name='activation')

        # Variable summaries
        variable_summary(weights, 'weights')
        if bias is not None:
            variable_summary(biases, 'biases')
        tf.summary.histogram('pre-activations', normalized)
        tf.summary.histogram('activations', activations)

        return activations

In [18]:
def gaussian_sample(mean, stddev, name):

    with tf.variable_scope(name):

        # mean is unconstrained; stddev must be strictly positive
        stddev = 1e-6 + tf.nn.softplus(stddev)

        # actually sample
        z = mean + stddev * tf.random_normal(tf.shape(mean), 0, 1, dtype=tf.float32)

        # Variable summaries
        tf.summary.histogram('mean', mean)
        tf.summary.histogram('stddev', stddev)
        tf.summary.histogram('z', z)

        return mean, stddev, z

In [19]:
def concatenate(x, label, train, name='cat'):
    
    with tf.variable_scope(name):
        
        cat = tf.concat([x, label], 1)
        
        # Variable summaries
        tf.summary.histogram('cat', cat)
        return cat

## Loss

In [20]:
def evaluate(x, xhat, mu, sigma):

    with tf.variable_scope('loss'):
        # Structure
        pred = tf.losses.absolute_difference(x, xhat,
                reduction=tf.losses.Reduction.MEAN)

        # offsetx = x + 0.5
        # pred = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        #         labels=offsetx, logits=xhat))

        KLdiv = 0.5 * tf.reduce_mean(tf.square(mu) + \
                    tf.square(sigma) - tf.log(1e-8 + tf.square(sigma)) - 1)

        loss = tf.add(REGULARIZATION * KLdiv, pred)

        # Summaries
        tf.summary.scalar('prediction', pred)
        tf.summary.scalar('prior', KLdiv)
        tf.summary.scalar('loss', loss)

    return loss, pred, KLdiv

## Autoencoder

In [21]:
def encoder(img, label, train):

    with tf.variable_scope('encoder'):

        label = tf.to_float(label)
        
        # convolution
        conv1 = convolution_layer(img, [5, 5, 1, 64], 2, train, name='conv1')
        conv2 = convolution_layer(conv1, [5, 5, 64, 128], 2, train, name='conv2')
        conv3 = convolution_layer(conv2, [5, 5, 128, 256], 2, train, name='conv3')

        # transition
        conv3 = tf.reshape(conv3, [-1, 32*32*256], name='reshape1')

        # dense output
        fc1_label = dense_layer(label, [15, 32], train,
                activation=tf.identity, name='fc1_label')     
        
        # concatenate output with labels (CVAE)
        cat1 = concatenate(conv3, fc1_label, train, name='cat1')
        
        # dense output
        fc2_mu = dense_layer(cat1, [32*32*256+32, 32], train,
                activation=tf.identity, name='fc2_mu')
              
        # dense output
        fc3_sigma = dense_layer(conv3, [32*32*256, 32], train,
                activation=tf.identity, name='fc3_sigma')
        # sample
        mu, sigma, z = gaussian_sample(fc2_mu, fc3_sigma, name='output')

    return mu, sigma, z

In [22]:
def decoder(z, label, train):

    with tf.variable_scope('decoder'):

        label = tf.to_float(label)
        
        # dense input
        fc1_label = dense_layer(label, [15, 2*32], train, name='fc1_label')
        
        # concatenate output with labels (CVAE)
        cat1 = concatenate(z, fc1_label, train, name='cat1')
        
        # dense input
        fc1 = dense_layer(cat1, [32+(2*32), 32*32*256], train, name='fc1')

        # transition
        fc1 = tf.reshape(fc1, [-1, 32, 32, 256], name='reshape1')

        # deconvolution
        deconv1 = deconvolution_layer(fc1, [5, 5, 256, 128], 2, train,
                            name='deconv1')
        deconv2 = deconvolution_layer(deconv1, [5, 5, 128, 64], 2, train, 
                            name='deconv2')
        deconv3 = deconvolution_layer(deconv2, [5, 5, 64, 32], 2, train,
                            name='deconv3')
        logits = deconvolution_layer(deconv3, [5, 5, 32, 1], 1, train,
                            activation=tf.identity, name='logits')

        # put into image range for display
        with tf.name_scope('range'):
            xhat = 0.5 * tf.nn.tanh(logits)

    return xhat, logits

# Build and run

## Initialize

In [23]:
import sys
import os
import argparse
from datetime import datetime

import logging
logging.basicConfig(
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    format="%(asctime)s [train/initialize] %(levelname)-8s %(message)s",
    level=logging.INFO
)

NOW_STR = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S%z")
RUN_DESC = "cross-entropy loss, 256x256 images, no bias"
RANDOM_SEED = 42


In [26]:
def initialize():

    logging.info("initializing run: {}".format(RUN_NAME))

    # write a note regarding this run
    os.makedirs(SUMMARY_DIR, exist_ok=True)
    with open(os.path.join(SUMMARY_DIR, "description.txt"), 'w') as fh:
        fh.write(NOW_STR+" "+RUN_DESC)

    # Control memory usage
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    # Report OOM details
    run_options = tf.RunOptions(report_tensor_allocations_upon_oom = True)

    # Build graph
    with tf.Graph().as_default() as graph:

        # Repeatable results
        tf.set_random_seed(RANDOM_SEED)

        # Get Data
        train_dataset, train_iterator = inputs(filenames=TRAIN_TFRECORDS,
                batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, num_shards = 1, shard_index = 0)

        # Data placeholder
        data_handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            data_handle, train_dataset.output_types, train_dataset.output_shapes)
        
       

        filename, image, labels = iterator.get_next()

        # Train/validate flag
        train = tf.placeholder(tf.bool)

        # Global counter
        global_step = tf.train.get_or_create_global_step(graph)

        # Dropout
        dropout = tf.placeholder(tf.float32)

        # Autoencoder
        mu, sigma, z = encoder(image, labels, train)
        xhat, logits = decoder(z, labels, train)
        loss, recon, reg = evaluate(image, xhat, mu, sigma)

        # Training branch - control dependencies so batchnorm params are updated
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)\
                .minimize(loss, global_step=global_step, name='optimizer')

        # Log output for Tensorboard
        merged = tf.summary.merge_all()
        train_summary_logger = tf.summary.FileWriter(SUMMARY_DIR+'/train',
                        graph=graph, flush_secs=30)

        # Initializer
        init_variables = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

        # Save state
        tf.add_to_collection('optimizer', optimizer)

        tf.add_to_collection('filename', filename)
        tf.add_to_collection('image', image)
        #tf.add_to_collection('view', view)

        #tf.add_to_collection('gender', gender)
        #tf.add_to_collection('age', age)
        tf.add_to_collection('labels', labels)
        
        tf.add_to_collection('mu', mu)
        
        tf.add_to_collection('sigma', sigma)
        tf.add_to_collection('xhat', xhat)

        tf.add_to_collection('loss', loss)
        tf.add_to_collection('recon', recon)
        tf.add_to_collection('reg', reg)

        tf.add_to_collection('data_handle', data_handle)
        tf.add_to_collection('train', train)

        tf.add_to_collection('merged', merged)

        writer = tf.train.Saver()

        # Run one step: this initializes the graph and saves our starting statistics
        with tf.Session(config=config) as session:

            session.run(init_variables)

            train_handle = session.run(train_iterator.string_handle())

            # Output header
            logging.info("  step      loss      recon     reg")

            _, step = session.run([optimizer, global_step],
                    feed_dict = { data_handle: train_handle, train: 1 },
                    options = run_options)

            loss_, recon_, reg_, summary = \
                session.run([loss, recon, reg, merged],
                           feed_dict = { data_handle: train_handle, train: 0 })
            train_summary_logger.add_summary(summary, step)

            logging.info("{: 6d} {:9.3g} {:9.3g} {:9.3g}".format(step, loss_, recon_, reg_))

            # Save graph
            logging.info("saving graph")
            writer.save(session, MODEL_PREFIX, global_step=step, write_meta_graph=False)
            writer.export_meta_graph(MODEL_GRAPH)
            


def main():

    parser = argparse.ArgumentParser(description='initialize training graph')
    initialize()
    sys.exit(0)


if __name__ == '__main__':
    main()

2018-07-11T23:29:09+0000 [train/initialize] INFO     initializing run: cvae/vae_004
2018-07-11T23:29:25+0000 [train/initialize] INFO       step      loss      recon     reg
2018-07-11T23:29:41+0000 [train/initialize] INFO          0     0.209     0.198     0.104
2018-07-11T23:29:41+0000 [train/initialize] INFO     saving graph


SystemExit: 0

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Train

In [27]:
with tf.Graph().as_default() as graph:
    
    # Repeatable results
    tf.set_random_seed(0)

    # Get Data
    _, train_iterator = inputs(filenames=TRAIN_TFRECORDS,
                batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, num_shards = 1, shard_index = 0)

        
    # Log output for Tensorboard
    train_summary_logger = tf.summary.FileWriter(SUMMARY_DIR+'/train', flush_secs=30)

    # Run
    with tf.Session(config=config) as session:

        # restore
        reader = tf.train.import_meta_graph(MODEL_GRAPH)
        reader.restore(session, tf.train.latest_checkpoint(MODEL_DIR))

        # must be called after reader so that the graph is populated
        writer = tf.train.Saver()

        # get references to graph endpoints
        global_step = tf.train.get_global_step(graph)

        optimizer = tf.get_collection('optimizer')[0]
        loss = tf.get_collection('loss')[0]
        recon = tf.get_collection('recon')[0]
        reg = tf.get_collection('reg')[0]

        data_handle = tf.get_collection('data_handle')[0]
        train = tf.get_collection('train')[0]

        merged = tf.get_collection('merged')[0]

        train_handle = session.run(train_iterator.string_handle())

        # Output header
        logging.info("  step      loss      recon     reg")

        while True:
            try:
                _, step = session.run([optimizer, global_step], 
                            feed_dict = { data_handle: train_handle, train: 1 })

                if not step%DISPLAY_EVERY:

                    loss_, recon_, reg_, summary = session.run([loss, recon, reg, merged],
                            feed_dict = { data_handle: train_handle, train: 0 })
                    train_summary_logger.add_summary(summary, step)

                    logging.info("{: 6d} {:9.3g} {:9.3g} {:9.3g}".format(
                        step, loss_, recon_, reg_))

                    writer.save(session, MODEL_PREFIX, global_step=step, write_meta_graph=False)

            except tf.errors.OutOfRangeError:
                print("done")
                break

INFO:tensorflow:Restoring parameters from ../../data/models/cvae/vae_004/vae-0


2018-07-11T23:29:49+0000 [train/initialize] INFO     Restoring parameters from ../../data/models/cvae/vae_004/vae-0
2018-07-11T23:29:50+0000 [train/initialize] INFO       step      loss      recon     reg
2018-07-11T23:30:37+0000 [train/initialize] INFO        100     0.365     0.346     0.194
2018-07-11T23:31:25+0000 [train/initialize] INFO        200     0.395     0.352     0.438
2018-07-11T23:32:14+0000 [train/initialize] INFO        300     0.381     0.352     0.286
2018-07-11T23:33:02+0000 [train/initialize] INFO        400     0.381     0.349     0.316
2018-07-11T23:33:51+0000 [train/initialize] INFO        500     0.373     0.336      0.37
2018-07-11T23:34:39+0000 [train/initialize] INFO        600     0.392     0.303      0.89
2018-07-11T23:35:27+0000 [train/initialize] INFO        700     0.363     0.283     0.801
2018-07-11T23:36:15+0000 [train/initialize] INFO        800     0.405     0.274      1.31
2018-07-11T23:37:04+0000 [train/initialize] INFO        900     0.365     0

2018-07-12T00:44:42+0000 [train/initialize] INFO       9000     0.139     0.132    0.0751
2018-07-12T00:45:28+0000 [train/initialize] INFO       9100      0.18      0.17     0.102
2018-07-12T00:46:16+0000 [train/initialize] INFO       9200      0.17     0.163    0.0754
2018-07-12T00:47:04+0000 [train/initialize] INFO       9300     0.173     0.168    0.0537
2018-07-12T00:47:52+0000 [train/initialize] INFO       9400     0.164      0.16    0.0468
2018-07-12T00:48:39+0000 [train/initialize] INFO       9500      0.16     0.156    0.0388
2018-07-12T00:49:28+0000 [train/initialize] INFO       9600     0.175     0.167     0.079
2018-07-12T00:50:16+0000 [train/initialize] INFO       9700     0.184     0.177     0.073
2018-07-12T00:51:04+0000 [train/initialize] INFO       9800     0.177     0.172    0.0484
2018-07-12T00:51:52+0000 [train/initialize] INFO       9900     0.172     0.169    0.0358
2018-07-12T00:52:40+0000 [train/initialize] INFO      10000     0.171     0.168    0.0308
2018-07-12

2018-07-12T01:58:18+0000 [train/initialize] INFO      18200      0.18     0.177    0.0318
2018-07-12T01:59:06+0000 [train/initialize] INFO      18300     0.181     0.177    0.0422
2018-07-12T01:59:54+0000 [train/initialize] INFO      18400     0.177     0.175    0.0292
2018-07-12T02:00:43+0000 [train/initialize] INFO      18500     0.164     0.162    0.0253
2018-07-12T02:01:31+0000 [train/initialize] INFO      18600     0.182     0.179     0.027
2018-07-12T02:02:19+0000 [train/initialize] INFO      18700     0.187     0.184    0.0303
2018-07-12T02:03:07+0000 [train/initialize] INFO      18800     0.192      0.19    0.0262
2018-07-12T02:03:55+0000 [train/initialize] INFO      18900     0.183      0.18    0.0334
2018-07-12T02:04:43+0000 [train/initialize] INFO      19000     0.185     0.182    0.0293
2018-07-12T02:05:33+0000 [train/initialize] INFO      19100     0.176     0.173     0.035
2018-07-12T02:06:21+0000 [train/initialize] INFO      19200     0.158     0.155    0.0241
2018-07-12

2018-07-12T03:12:02+0000 [train/initialize] INFO      27400     0.172     0.169    0.0309
2018-07-12T03:12:50+0000 [train/initialize] INFO      27500     0.164     0.162    0.0275
2018-07-12T03:13:38+0000 [train/initialize] INFO      27600     0.156     0.153    0.0259
2018-07-12T03:14:27+0000 [train/initialize] INFO      27700     0.172     0.168     0.034
2018-07-12T03:15:15+0000 [train/initialize] INFO      27800     0.182     0.179    0.0353
2018-07-12T03:16:03+0000 [train/initialize] INFO      27900     0.188     0.185    0.0321
2018-07-12T03:16:51+0000 [train/initialize] INFO      28000     0.176     0.173    0.0266
2018-07-12T03:17:39+0000 [train/initialize] INFO      28100     0.166     0.163    0.0309
2018-07-12T03:18:27+0000 [train/initialize] INFO      28200     0.175     0.173    0.0272
2018-07-12T03:19:15+0000 [train/initialize] INFO      28300     0.179     0.176    0.0285
2018-07-12T03:20:03+0000 [train/initialize] INFO      28400      0.17     0.166    0.0344
2018-07-12

2018-07-12T04:25:54+0000 [train/initialize] INFO      36600     0.173      0.17    0.0289
2018-07-12T04:26:42+0000 [train/initialize] INFO      36700     0.163     0.161    0.0252
2018-07-12T04:27:30+0000 [train/initialize] INFO      36800     0.174     0.171    0.0329
2018-07-12T04:28:19+0000 [train/initialize] INFO      36900     0.186     0.182    0.0403
2018-07-12T04:29:07+0000 [train/initialize] INFO      37000     0.187     0.182    0.0408
2018-07-12T04:29:55+0000 [train/initialize] INFO      37100     0.164     0.161    0.0322
2018-07-12T04:30:42+0000 [train/initialize] INFO      37200     0.172     0.169    0.0302
2018-07-12T04:31:31+0000 [train/initialize] INFO      37300     0.157     0.153    0.0377
2018-07-12T04:32:19+0000 [train/initialize] INFO      37400     0.175     0.172    0.0358
2018-07-12T04:33:07+0000 [train/initialize] INFO      37500     0.163     0.159    0.0404
2018-07-12T04:33:56+0000 [train/initialize] INFO      37600      0.17     0.167    0.0328
2018-07-12

2018-07-12T05:39:37+0000 [train/initialize] INFO      45800     0.183     0.179    0.0421
2018-07-12T05:40:25+0000 [train/initialize] INFO      45900     0.185     0.181    0.0391
2018-07-12T05:41:13+0000 [train/initialize] INFO      46000     0.194      0.19    0.0397
2018-07-12T05:42:01+0000 [train/initialize] INFO      46100      0.18     0.176    0.0445
2018-07-12T05:42:50+0000 [train/initialize] INFO      46200     0.163     0.159    0.0415
2018-07-12T05:43:38+0000 [train/initialize] INFO      46300     0.167     0.163    0.0379
2018-07-12T05:44:26+0000 [train/initialize] INFO      46400     0.169     0.165     0.045
2018-07-12T05:45:14+0000 [train/initialize] INFO      46500     0.168     0.164    0.0401
2018-07-12T05:46:01+0000 [train/initialize] INFO      46600     0.167     0.163    0.0395
2018-07-12T05:46:49+0000 [train/initialize] INFO      46700     0.176     0.172    0.0403
2018-07-12T05:47:38+0000 [train/initialize] INFO      46800     0.172     0.169    0.0349
2018-07-12

2018-07-12T06:53:14+0000 [train/initialize] INFO      55000     0.174      0.17    0.0424
2018-07-12T06:54:02+0000 [train/initialize] INFO      55100     0.171     0.166    0.0508
2018-07-12T06:54:50+0000 [train/initialize] INFO      55200     0.167     0.163    0.0394
2018-07-12T06:55:38+0000 [train/initialize] INFO      55300     0.159     0.155    0.0377
2018-07-12T06:56:27+0000 [train/initialize] INFO      55400     0.169     0.165    0.0395
2018-07-12T06:57:15+0000 [train/initialize] INFO      55500     0.155     0.151    0.0421
2018-07-12T06:58:03+0000 [train/initialize] INFO      55600     0.167     0.162    0.0442
2018-07-12T06:58:52+0000 [train/initialize] INFO      55700     0.163     0.159    0.0354
2018-07-12T06:59:39+0000 [train/initialize] INFO      55800     0.174      0.17    0.0395
2018-07-12T07:00:28+0000 [train/initialize] INFO      55900     0.171     0.168    0.0375
2018-07-12T07:01:16+0000 [train/initialize] INFO      56000      0.17     0.166     0.036
2018-07-12

2018-07-12T08:07:03+0000 [train/initialize] INFO      64200     0.157     0.152    0.0505
2018-07-12T08:07:50+0000 [train/initialize] INFO      64300     0.174      0.17    0.0426
2018-07-12T08:08:38+0000 [train/initialize] INFO      64400     0.166     0.161    0.0486
2018-07-12T08:09:26+0000 [train/initialize] INFO      64500     0.168     0.164    0.0383
2018-07-12T08:10:14+0000 [train/initialize] INFO      64600     0.163     0.159    0.0401
2018-07-12T08:11:02+0000 [train/initialize] INFO      64700     0.154      0.15    0.0398
2018-07-12T08:11:50+0000 [train/initialize] INFO      64800     0.165     0.161    0.0468
2018-07-12T08:12:38+0000 [train/initialize] INFO      64900     0.157     0.153    0.0396
2018-07-12T08:13:27+0000 [train/initialize] INFO      65000     0.171     0.167    0.0331
2018-07-12T08:14:15+0000 [train/initialize] INFO      65100     0.164      0.16    0.0378
2018-07-12T08:15:03+0000 [train/initialize] INFO      65200     0.162     0.157    0.0507
2018-07-12

done


# Test

Try out the autoencoder by running it on some test set samples.

For a collection of test images:

1. Create (image, label, mu, sigma) tuples
1. For a seed image, compute the 10 nearest images using (mu, sigma)
1. View the nearby images and their labels, comparing them to the seed image

If the VAE has worked as expected, we should find that the nearby images match the seed visually, and perhaps even match according to their labels.

## Generate document vectors

In [28]:


def gen_documents(tfrecords, dir_name, shard_index):
    
    # Store the documents
    documents = []
    def extend(docs, m, s, xh, f, i, v, a, g, l):
        start_id = len(docs)
        docs.extend([
            {
                'filename':f_.decode('ascii') ,
                'id_': k + start_id,
                'image': i_.reshape(256, 256)+0.5,
                'view': v_.decode('ascii') ,
                'age': a_,
                'gender': g_.decode('ascii'),
                'labels': l_,
                'sigma': s_,
                'mu': m_,
                'xhat': y_.reshape(256, 256)+0.5
            }
            for k, (m_, s_, y_, f_, i_, v_, a_, g_, l_) in enumerate(zip(m, s, xh, f, i, v, a, g, l))
        ])
        return docs

    with tf.Graph().as_default() as graph:

        # Repeatable results
        tf.set_random_seed(0)

        # Get Data
        dataset, iterator = inputs(filenames=tfrecords,
                batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, num_shards = 1, shard_index = 0)
        # Log output for Tensorboard
        summary_logger = tf.summary.FileWriter(SUMMARY_DIR + dir_name, flush_secs=30)

        print(SUMMARY_DIR + dir_name)
        # Run
        with tf.Session(config=config) as session:

            # restore
            reader = tf.train.import_meta_graph(MODEL_GRAPH)
            reader.restore(session, tf.train.latest_checkpoint(MODEL_DIR))

            # get references to graph endpoints
            filename = tf.get_collection('filename')[0]
            image = tf.get_collection('image')[0]
            #view = tf.get_collection('view')[0]
            #age = tf.get_collection('age')[0]
            #gender = tf.get_collection('gender')[0]
            labels = tf.get_collection('labels')[0]

            mu = tf.get_collection('mu')[0]
            sigma = tf.get_collection('sigma')[0]
            xhat = tf.get_collection('xhat')[0]

            merged = tf.get_collection('merged')[0]

            data_handle = tf.get_collection('data_handle')[0]
            train = tf.get_collection('train')[0]
            handle = session.run(iterator.string_handle())
            
            step = 0
            while True:
                try:
                    mu_, sigma_, xhat_, filename_, image_, labels_, summary = \
                        session.run([mu, sigma, xhat, filename, image, labels, merged],
                                feed_dict = { data_handle: handle, train: 0 })
                    documents = extend(documents, mu_, sigma_, xhat_, filename_, image_, labels_)

                    step += 1
                    summary_logger.add_summary(summary, step)                        
                    print("{}.".format(step), end="", flush=True)

                except tf.errors.OutOfRangeError:
                    print(".done")
                    break
    return documents

In [29]:
SUBSET = "golden"

In [30]:
if SUBSET == "golden":
    golden_documents = gen_documents(GOLDEN_TFRECORDS, '/golden', 1)
elif SUBSET == "validate":
    validate_documents = gen_documents(VALIDATE_TFRECORDS, '/validate')
elif SUBSET == "test":
    test_documents = gen_documents(TEST_TFRECORDS, '/test')
elif SUBSET == "train":
    train_documents = gen_documents(TRAIN_TFRECORDS, '/train', 3)


../../data/logs/cvae/vae_004/golden
INFO:tensorflow:Restoring parameters from ../../data/models/cvae/vae_004/vae-67800


2018-07-11T19:18:25+0000 [train/initialize] INFO     Restoring parameters from ../../data/models/cvae/vae_004/vae-67800


1..done


## Generate Json

In [34]:
import json
import scipy.misc as misc

In [32]:
def gen_json(documents):
    
    for doc in documents:
        json_item = {
            'id':doc['id_'],
            'filename': doc['filename'],
            'subset': SUBSET,
            'encodedname': doc['filename'],
            'labels': doc['labels'].tolist(),
            'age': int(doc['age']),
            'gender': doc['gender'],
            'view': doc['view'],
            'mu':doc['mu'].tolist(),
            'sigma':doc['sigma'].tolist()
        }

        JSON_DIR = os.path.join(EMBEDDINGS_DIR, doc['filename'][:12] + ".json")
        with open(JSON_DIR, 'w') as outfile:
            json.dump(json_item, outfile, separators=(',', ':'), indent = 2)
    

In [ ]:
if SUBSET == "golden":
    gen_json(golden_documents)

elif SUBSET == "validate":
    gen_json(validate_documents)
    
elif SUBSET == "test":
    gen_json(test_documents)
    
elif SUBSET == "train":
    gen_json(train_documents)

## Save Images

In [35]:

#save the encoded 256x256 vectors as png file
def save_images(documents):
    for doc in documents:

        path = os.path.join(ENCODED_IMAGES_DIR, doc['filename'])
        misc.imsave(path, doc['xhat'])


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [ ]:
if SUBSET == "golden":
    save_images(golden_documents)

elif SUBSET == "validate":
    save_images(validate_documents)
    
elif SUBSET == "test":
    save_images(test_documents)
    
elif SUBSET == "train":
    save_images(train_documents)